In [1]:
# imports
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np

In [2]:
driver = webdriver.Chrome("/Applications/chromedriver")
url = "https://ivyleague.com/calendar.aspx?path=mhockey"
driver.get(url)
content = driver.page_source
soup = BeautifulSoup(content)

In [3]:
results = []
main = soup.find('main')
tables = main.find('div', attrs={'class' : 'sidearm-calendar-list'})
for table in tables.findAll('table', attrs={'class' : 'sidearm-calendar-list-group sidearm-calendar-table'}):
    date_str = table.find('caption').find('span').findAll(text=True)[0]
    date = datetime.strptime(date_str, '%A, %B %d, %Y')
    rows = table.find('tbody')
    for row in rows.findAll('tr'):
        result = {'date' : date}
        cells = row.findAll('td')
        for i in [1,2]:   
            team = (cells[i].find('span', attrs={'class': 'sidearm-calendar-list-group-list-game-team-title'})
                            .find('a').findAll(text=True)[0])
            try:
                score = (cells[i].find('span', attrs={'class': 
                                                  'sidearm-calendar-list-group-list-game-team-score text-no-wrap'})
                                 .findAll(text=True)[0])
            except:
                print(date_str, team)
                score = input('score: ')
            result[i] = dict(team=team, score=score)
        results.append(result)

Thursday, February 27, 2020 Cornell
score: 3
Thursday, February 27, 2020 Dartmouth
score: 1


In [4]:
cleaned_results = []
for result in results:
    cleaned_result = {}
    cleaned_result['date'] = result['date']
    cleaned_result['team_1'] = result[1]['team']
    cleaned_result['score_1'] = int(result[1]['score'])
    cleaned_result['team_2'] = result[2]['team']
    cleaned_result['score_2'] = int(result[2]['score'])
    if cleaned_result['score_1'] > cleaned_result['score_2']:
        cleaned_result['winner'] = cleaned_result['team_1']
    elif cleaned_result['score_2'] > cleaned_result['score_1']:
        cleaned_result['winner'] = cleaned_result['team_2']
    else:
        cleaned_result['winner'] = 'tie'
    cleaned_results.append(cleaned_result)

In [5]:
results = pd.DataFrame(cleaned_results)
teams = list(set(list(results['team_1'].unique()) + list(results['team_2'].unique())))
id_to_teams = teams
teams_to_id = {id_to_teams[i] : i for i in range(len(teams))}

In [8]:
def instance(start, now, end):
    past = results[(results.date >= start) & (results.date <= now)]
    future = results[(results.date > now) & (results.date <= end)]
    w = {}
    for team in teams:
        w[team] = 2*len(past[past.winner == team])
    for index, row in past[past.winner == 'tie'].iterrows():
        w[row['team_1']] = w[row['team_1']] + 1
        w[row['team_2']] = w[row['team_2']] + 1
    w = pd.DataFrame(w, index=[0]).transpose().rename(columns={0 : 'w'})
    
    games_left = np.zeros((len(teams), len(teams)))
    for i in teams:
        for j in teams:
            rem = len((future[((future.team_1 == i) & (future.team_2 == j)) |
                              ((future.team_1 == j) & (future.team_2 == i))]))
            games_left[teams_to_id[i], teams_to_id[j]] = rem
            games_left[teams_to_id[j], teams_to_id[i]] = rem
    games_left = pd.DataFrame(index=teams, columns=teams, data=games_left).astype(int)
            
    return (w, games_left)

In [9]:
w, games_left = instance(datetime(2019, 11, 1), datetime(2019, 11, 3), datetime(2020, 3, 8))

In [10]:
w

,w
Colgate,0
Yale,2
Union,0
Quinnipiac,0
Michigan State,0
UConn,0
New Hampshire,2
Boston University,0
Dartmouth,0
Clarkson,0


In [11]:
games_left

,Colgate,Yale,Union,Quinnipiac,Michigan State,UConn,New Hampshire,Boston University,Dartmouth,Clarkson,...,American International,Rensselaer,Harvard,St. Lawrence,Colorado College,Northeastern,St. Cloud State,Brown,Ohio State,Boston College
Colgate,0,2,0,0,0,0,0,0,2,0,...,0,0,2,0,0,0,0,4,0,0
Yale,2,0,5,2,0,1,1,0,2,2,...,0,2,3,2,0,0,0,0,0,1
Union,0,5,0,0,0,0,0,0,2,0,...,0,0,2,0,0,0,0,2,0,0
Quinnipiac,0,2,0,0,0,0,0,0,2,0,...,0,0,2,0,0,0,0,2,0,0
Michigan State,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
UConn,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
New Hampshire,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Boston University,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
Dartmouth,2,2,2,2,0,1,0,1,0,2,...,0,2,1,2,1,1,0,2,0,0
Clarkson,0,2,0,0,0,0,0,0,2,0,...,0,0,2,0,0,0,0,2,0,0
